In [18]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance, ImageOps
import PIL
import os
import easyocr
import openpyxl
import cv2
import re
import streamlit as st

<img src='anchor.png'>

# Якорь
<p>Принцип работы: <br>
После загрузки файла на сервер следует обработка - удаляются шумы, увеличивается контраст, выделяются линии - это позволяет модели EasyOCR лучше распознавать текст. 
<br>Дальше выбираем, какую модель мы будем использовать: русскую для вк, ютуба и дзена и английскую для телеграмма. Всё это делается, чтобы модель могла найти якорь-ключевое слово. <br>
    Если ключевое слово найдено, алгоритм обрезает изображение в соответствии с паттером соцсети (у каждой соцсети свой паттерн поведения). Это нужно, чтобы убрать лишнюю информацию и облегчить работу по поиску KPI с помощью регулярных выражений и встроенных алгоритмов. <br>
    Обрезанное изображение снова отправляет в модель для поиска KPI. Модель возвращает минимальный список со строками, которые в последствии мы разбираем, чтобы вытащить нужные данные.<br>
    
В случае, если якорь не был найдет (нечитаемый скриншот или отсутствует нужная информация) в таблицу добавляется запись с именем файла и пометкой "Требуется ручная проверка>

# От команды AI-Wizard
<p>
Отдельное спасибо организаторам, техническому-эксперту Ринату и нашему трекеру Елене. Для нашей команды данный кейс был действительно сложным - в нашей команде не было ни одного человека, который до этого работал бы с компьютерным зрением, пришлось учиться на ходу. И это сильно помогло нам сплотиться ради достижения цели. <br>
От себя лично добавлю, я рад, что у нас получилось завершить проект. Не бросили, не испугались. Пытались, ошибались (много ошибались, очень много), но результат есть и это очень приятно. <br>

Что не было сделано: <br>

EasyOCR может расширяться за счёт других моделей - это сильно повышает её эффективность. На это просто не было времени, ни у кого не было возможности потратить время на то, чтобы понять как эти модели подключать и как две модели должны взаимодействовать. <br>
Этапы обработки - следует внедрить в модель несколько этапов обработки изображения, чтобы лучше обрабатывать текст. В процессе прогонки возникало ощущение, что некоторые цифры просто заколдованные - ни при каких условиях модель их не замечала. Были испробаваны разные варианты: удаление шумов, повышение контраста, изменение расширения файла, удаление альфа-канала. И ничего не принесло результата. А время было потрачено. <br>
Как аналитики, ml-инженеры и data science мы должны были предоставить несколько моделей для решения проблемы. Только сталкивая разные модели, можно получить хороший результат. Где-то объединять модели (ансамбли), где-то строить нейронные сети из разных типов слоёв (RNN + LSTM). Не хватило опыта и времени. <br>
Кейс очень крутой, мы довольны. <br>
Спасибо Вам.
</p>
<img src='AI_Wizard.jpg' height=200 width=200><br>
(с) Данный логотип был сделан нашим дизайнером Даниилом и нейронной сетью Dream

# Анастасия
<p>Анастасия буквально спасла нашу команду - у нас не было опыта разработки UI. Несмотря на то, что задача для Анастасии была в новинку, она успешно её решила создав UI с помощью web-технологий и Python.</p>

In [22]:
text = pd.DataFrame(columns=['Файл', 'Успешная загрузка', 'Метрика 1', 'Метрика 2'])

#функция для получения пути файла
def get_image_path(img):
    file_path = f"data/uploadedImages/{img.name}"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "wb") as img_file:
        img_file.write(img.getbuffer())
    return file_path


st.set_page_config(layout='centered', initial_sidebar_state='auto')

image = Image.open('AI_Wizard.jpg')
st.image(image, caption='', width=100, output_format='JPEG')

st.title('Сбор статистики')

uploaded_files = st.file_uploader('Выберите фото', accept_multiple_files=True, type=['jpg', 'jpeg', 'png'])

st.selectbox('Выберите социальную сеть', ['VK', 'TG', 'Дзен', 'YouTube'], key='sn')
#soc_net = st.session_state.sn


#обработка входящих файлов
if uploaded_files is not None:
    if st.button('Готово'):
        for uploaded_file in uploaded_files:
            file_name = uploaded_file.name

            # Обработка изображения с помощью функции process_image
            # вызываемм функцию которая вернёт метрики (если их 2, если нет метрику + пустую строку)
            image_path = get_image_path(uploaded_file)

            try:
                metricOne, metricTwo = cut_at_anchor(image_path, soc_net)
            except:
                st.text(str(file_name) + ' повреждён')

            if (load == 1):
                st.text(str(file_name) + '\t' + 'Успешно загружен')
                text.loc[len(text)] = [file_name, 'Успешно загружен', '-']
            elif (load == 0):
                st.text(str(file_name) + '\t' + 'Требуется ручная проверка')
                text.loc[len(text)] = [file_name, 'Требуется ручная проверка', '-']

        if st.button('Скачать итоговый файл'):
            output = io.BytesIO()
            writer = pd.ExcelWriter(output, engine='xlsxwriter')
            text.to_excel(writer, index=False)
            writer.save()
            output.seek(0)

# Юрий 
<p>Юре тоже досталось: попробуй разбери список вида <br><br>0,dzen,0cdea12d-b73c-40cb-a5ee-3ae985ce862d.png,"['32 мая', '35 ная', '#аР', '20 мая', '26 ная', '31 мая', 'ЗЕГСЛЕ БС <', 'Дата !', 'Псказы', ""Д:ч!'"", 'Ыв: +7', ""~51е: вре;'я"", 'Пс:Г ,15К+1', '-р~сашстзы', 'Г-#-61:-/за', 'Всего за 30 дней', '672', '727', '173 ч-', '232', '164 публикации']". <br><br>А ведь тут спрятаны те самые заветные KPI, ради которых была вся задача.
</p>

In [ ]:
def exctract_metric(text_lists, social_net):
    """
    text_lists : list Список кортежей, содержащих слова и их координаты на изображении
    social_net : str Принимает значения: vk, tg, zn, yt
    """
    # Извлекаем только слова и переводим их к нижнему регистру
    text = [text_list[1].lower() for text_list in text_lists]

    # Опредлеяем метрику в зависимости от соц. сети
    if social_net == 'vk':
        # ищем кол-во подписчиков
        pattern = r"([\d ]*)(подписчик[ов]*)"
        # Извлекаем нужную метрику
        for id, word in enumerate(text):
            finded = re.findall(pattern, word)
            # нашлось ли совпадение с паттерном
            if len(finded) > 0:
                if finded[0][0] != '':
                    metric = finded[0][0]
                else:
                    try: metric = text[id-1]
                    except: metric = -1
            else: metric = -1

    elif social_net == 'tg':
        # ищем visibility rate
        pattern = r"^(ERR|VR)$"
        # Извлекаем нужную метрику
        for id, word in enumerate(text):
            finded = re.findall(pattern, word)
            # нашлось ли совпадение с паттерном
            if len(finded) > 0:
                try: metric = text[id-1]
                except: metric = -1
            else: metric = -1

    elif social_net == 'zn':
        # ищем кол-во дочитываний
        pattern = r"(дней|день)$"
        # Извлекаем нужную метрику
        for id, word in enumerate(text):
            finded = re.findall(pattern, word)
            # нашлось ли совпадение с паттерном
            if len(finded) > 0:
                try: metric = text[id+2]
                except: metric = -1
            else: metric = -1

    elif social_net == 'yt':
        metric = -1
    #     # ищем кол-во подписчиков
    #     pattern1 = r"^(подписчик[и|ов]+)$"
    #     # Извлекаем нужную метрику
    #     for id, word in enumerate(text):
    #         finded = re.findall(pattern1, word)
    #         # нашлось ли совпадение с паттерном
    #         if len(finded) > 0:
    #             try: metric = text[id-1]
    #             except: metric = -1
    #         else: metric = -1
    #     # ищем кол-во просмотров за месяц
    #     pattern2 = r""
    
    return str(metric)

# Ирина (капитан) и Игорь
<p>Ну, а нам пришлось постигать азы компьютерного зрения. Мы перебрали 3 модели: 
<ul>
    <li>PaddleOCR - тяжёло настраивать, результат не впечатлил;</li>
    <li>tesseract - мы на него очень надеялись. Простой в освоении, но какой-же он привереда. Помучавшись часов 12 с нечитаемым текстом, забросили;</li>
    <li>EasyOCR - наш спаситель. Только эта модель "из коробки" доставала хоть сколько-нибудь читаемый текст. Потом была придумана идея с якорем;</li>
</ul>
Мы перебрали разные способы фильтрации и обработки изображения - opencv, PIL, Pillow. Что-то осталось, что-то выкидывалось и возвращалось обратно. Бегом, бегом успеть решить задачу. Подвела CUDA, которая не при каких условиях не хотела работать, а это потеря производительности - потеря времени. Чтобы понять, получилось у нас или нет, приходилось ждать иногда час. <br>
Конечный представленный вариант является скриптом, заходит в папку "temp" (на веб-сервере эта папка как раз используется для того, чтобы временно сохранить файлы - к примеру когда мы загружаем музыку или картинки в интернет) с параметром соцсеть - из нашего алгоритма это паттер того, что нужно делать. Скрипт проходит по всем файлам, достаёт из них список с словами (строки) и записывает его в pandas.DataFrame. После того, как цикл закончит свою работу и все файлы будут перебраны, происходит сохранение в файл xlsx. 
</p>

In [23]:
#всё в одной функции, чтобы не плодить сущности - создали Reader и гоняем его в рамках одной картинки

#триггер удачного нахождения якоря
load = 0

#надо ещё доработать эту функцию, мне кажется за счёт
#улучшения качества фото можно улучшить предикты модели
def cut_at_anchor(img_path, soc_net):
    #открываем изображение с помощью Pillow
    try:
        image = Image.open(img_path)
    except FileNotFoundError:
        print('Файл повреждён')

    cropped_image = image
    
    #создаём экземпляр сети
    if(soc_net!='tg'):
        reader = easyocr.Reader(['ru'])             
    else:
        reader = easyocr.Reader(['en'])

    #считываем размеры фото
    width, height = image.size

#Этап второй: ищем якорь и его позицию
#если якорь найти не удалось - выдаём ошибку ничего не записываем в pandas
    if(soc_net=='vk'):
        gray_image = image.convert('L')
        # Бинаризация изображения
        Threshold = 230
        result = gray_image.point(lambda x: 0 if x < Threshold else 255, 'L')
        image = result.filter(ImageFilter.SHARPEN)
        #threshold_vr = 100
        #binary_vr = image.point(lambda x: 0 if x < threshold_vr else 255)
        gray = np.asarray(image)
        result = reader.readtext(gray, batch_size=512, min_size=3)
        word_to_find = 'одписч'
        word_to_find2 = 'частник?'
        width, height = image.size
        
        for r in result:
            text = r[1]
            if word_to_find in text or word_to_find2 in text:
                load = 1
                #записываем позицию левого верхнего угла якоря
                top_pos = r[0][0]
                #обрезаем фото так: влево на 30 пикселей, вправо на 150 (сама надпись + растояние до цифры) 
                #и на высоту 50 (можно поиграть для точности)
                cropped_image = image.crop((top_pos[0]-width*0.05, top_pos[1]-height*0.1, top_pos[0]+width*0.2, top_pos[1]+height*0.1))
               
    elif(soc_net=='youtube'):
        word_to_find = 'подписчик'
        word_to_find2 = 'просмотра'
        word_to_find3 = 'подробнее'
        #подписчики => вверх и вниз обрезаем по 50 пикслей, если есть слово подробнее - оно становится главным якорем, над ним цифра
        #значит режем вверх
    elif(soc_net=='dzen'):
        #обрабатываем фото для лучшего распознавания
        #есть ещё над чем поработать
        #но уже хорошо работает
        width, height = image.size
        cropped_image = image.crop((0, height*0.7, width, height))
        gray_image = cropped_image.convert('L')
        # Бинаризация изображения
        Threshold = 190
        result = gray_image.point(lambda x: 0 if x < Threshold else 255, 'L')
        gray = np.asarray(result)
        word_to_find = 'день'
        word_to_find2 = 'дней'
        result.save('temp.png')
        result = reader.readtext(gray, batch_size=512, min_size=3, detail=0)
        return result
    else:
        #2 поиска - сначала ищем ERR, потом ищем VP
        gray_image = image.convert('L')
        #применяем разные маски для разных метрик
        threshold_err = 240
        binary_err = gray_image.point(lambda x: 0 if x < threshold_err else 255)
        threshold_vr = 170
        binary_vr = gray_image.point(lambda x: 0 if x < threshold_vr else 255)

        #прогоняем каждое изображение через одинаковые фильтры
        enhancer_err = ImageEnhance.Contrast(binary_err)
        enhancer_vr = ImageEnhance.Contrast(binary_vr)
        contrast_err= enhancer_err.enhance(3) # Увеличение контрастности в 3 раза
        contrast_vr = enhancer_vr.enhance(3) # Увеличение контрастности в 3 раза

        #якоря
        word_to_find = 'ERR'
        word_to_find2 = 'VR'

        #поиск якорей
        result_err = reader.readtext(np.asarray(contrast_err), batch_size=512, min_size=3)
        result_vr = reader.readtext(np.asarray(contrast_vr), batch_size=512, min_size=3)
        
        #если нашли якорь ERR => возвращаем его, так как 
        err_temp = ERR(result_err)
        if err_temp:
            return err_temp[1]

        #если не нашли, пытаемся найти VR
        for r in result_vr:
            text = r[1]
            if word_to_find2 in text:
                load = 1
                top_pos = r[0][0]
                cropped_image = contrast_vr.crop((top_pos[0], top_pos[1]+20, top_pos[0]+width*0.15, top_pos[1]+height*0.2))
    
    #если ничего не нашли => возвращаем пустую строку
    if cropped_image.size == image.size:
        return ''
    #если vr нашёлся, мы по обрезанной картинке ищем всего 1 значение - 
    result = reader.readtext(np.asarray(cropped_image), batch_size=512, min_size=5, detail=0)
    return result

In [282]:
# Путь к папке, в которой находятся папки с изображениями
root_folder = "temp"
df = pd.DataFrame(columns=['Соцсеть', 'Файл', 'Метрика'])

In [131]:
df.to_excel('vk.xlsx')

In [270]:
df

,Соцсеть,Файл,Метрика
0,dzen,0cdea12d-b73c-40cb-a5ee-3ae985ce862d.png,"[32 мая, 35 ная, #аР, 20 мая, 26 ная, 31 мая, ..."
1,dzen,0f00d72f-cf11-45d2-8051-2611bcbde777.png,"[35, #ня, Нян, 17, #нн, 23, ннн, 29 над, 0З ию..."
2,dzen,1a114fb6-9546-4087-9337-0bc5f1921baa.png,"[е-::, Дла, 7 = =а_, 2ъ - 'вва., ~_аата :, 2:,..."
3,dzen,1da9aa5e-ba13-411d-a6b8-bd18a24949b4.png,"[Заголовок, Дата ~, Пъказы, Дочитывания, Общев..."
4,dzen,2cda9a39-9c83-4e3e-ac79-6b0ad0358a42.jpg,"[#: СЛ:=СК, Дата !, Псказы, Азч:', ЫБ= +!'7, ~..."
5,dzen,3b7bce64-bff5-4dc4-8c49-645307a726ee.png,"[#, Ння, П, ннн, 14 ная, 20 ыая, 2Б ная, ЗЕГСЛ..."
6,dzen,3dda0209-e964-460d-8893-269fab3826fc.jpg,"[ЗаГолсвск, Дата, Показы, Дачить :а/ яя, Сбцее..."
